In [101]:
!pip install transformers torch pandas scikit-learn

In [102]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

In [103]:
#Load datasets

# train dataset
train_df = pd.read_csv("/Users/revatishelat/Desktop/dst/assessment 2/Data/medical_tc_train.csv")

# text dataset
test_df = pd.read_csv("/Users/revatishelat/Desktop/dst/assessment 2/Data/medical_tc_test.csv")

#labels 
labels_df = pd.read_csv("/Users/revatishelat/Desktop/dst/assessment 2/Data/medical_tc_labels.csv")

#convert dataframe to list

In [104]:
train_df

,condition_label,medical_abstract
0,5,Tissue changes around loose prostheses. A cani...
1,1,Neuropeptide Y and neuron-specific enolase lev...
2,2,"Sexually transmitted diseases of the colon, re..."
3,1,Lipolytic factors associated with murine and h...
4,3,Does carotid restenosis predict an increased r...
...,...,...
11545,1,Epirubicin at two dose levels with prednisolon...
11546,1,Four and a half year follow up of women with d...
11547,5,Safety of the transbronchial biopsy in outpati...
11548,3,Interictal spikes and hippocampal somatostatin...


In [105]:
test_df

,condition_label,medical_abstract
0,3,Obstructive sleep apnea following topical orop...
1,5,Neutrophil function and pyogenic infections in...
2,5,A phase II study of combined methotrexate and ...
3,1,Flow cytometric DNA analysis of parathyroid tu...
4,4,Paraneoplastic vasculitic neuropathy: a treata...
...,...,...
2883,4,Management of aortobronchial fistula with graf...
2884,5,Tuberculous psoas muscle abscess following che...
2885,4,Clinical evaluation of high-frequency (ultraso...
2886,1,Antitumor activity of liposome-encapsulated do...


In [106]:
labels

,condition_label,condition_name
0,1,neoplasms
1,2,digestive system diseases
2,3,nervous system diseases
3,4,cardiovascular diseases
4,5,general pathological conditions


In [107]:
train_texts = train_df["medical_abstract"] # 11550
train_labels = train_df["condition_label"]

test_texts = test_df["medical_abstract"]  # 2888
test_labels = test_df["condition_label"]

In [108]:
type(train_texts)
train_texts

0        Tissue changes around loose prostheses. A cani...
1        Neuropeptide Y and neuron-specific enolase lev...
2        Sexually transmitted diseases of the colon, re...
3        Lipolytic factors associated with murine and h...
4        Does carotid restenosis predict an increased r...
                               ...                        
11545    Epirubicin at two dose levels with prednisolon...
11546    Four and a half year follow up of women with d...
11547    Safety of the transbronchial biopsy in outpati...
11548    Interictal spikes and hippocampal somatostatin...
11549    Recurrent thoracic outlet syndrome after first...
Name: medical_abstract, Length: 11550, dtype: object

In [109]:
# train_labels

In [110]:
train_texts = train_df["medical_abstract"].astype(str).tolist() # 11550
train_labels = train_df["condition_label"].astype(int).tolist()

test_texts = test_df["medical_abstract"].astype(str).tolist()  # 2888
test_labels = test_df["condition_label"].astype(int).tolist()


In [111]:
type(train_texts)
# train_texts #outputs all teh text

list

In [112]:
type(test_texts)

list

In [113]:
# convert to list
# train_texts = train_texts.tolist()

# labels = df['label'].astype(int).tolist()

In [114]:
# type(train_texts)

In [115]:
# train_texts

In [116]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels)))

# Tokenize and encode the input text
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [127]:
# train_texts

In [118]:
train_encodings

{'input_ids': tensor([[  101,  8153,  3431,  ...,     0,     0,     0],
        [  101, 11265, 10976,  ...,     0,     0,     0],
        [  101, 12581, 11860,  ...,     0,     0,     0],
        ...,
        [  101,  3808,  1997,  ...,     0,     0,     0],
        [  101,  6970,  2594,  ...,     0,     0,     0],
        [  101, 28667, 29264,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [119]:
?train_labels

Type:        list
String form: [5, 1, 2, 1, 3, 3, 2, 4, 5, 5, 4, 1, 5, 1, 4, 2, 1, 4, 5, 1, 5, 1, 5, 1, 5, 2, 3, 4, 2, 1, 3, 4,  <...> , 5, 5, 5, 3, 4, 5, 1, 3, 1, 4, 4, 4, 1, 4, 3, 1, 2, 4, 5, 5, 3, 3, 5, 4, 5, 4, 5, 1, 1, 5, 3, 5]
Length:      11550
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [120]:
train_encodings

{'input_ids': tensor([[  101,  8153,  3431,  ...,     0,     0,     0],
        [  101, 11265, 10976,  ...,     0,     0,     0],
        [  101, 12581, 11860,  ...,     0,     0,     0],
        ...,
        [  101,  3808,  1997,  ...,     0,     0,     0],
        [  101,  6970,  2594,  ...,     0,     0,     0],
        [  101, 28667, 29264,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [121]:
# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

In [122]:
# Split the training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [123]:
# Create DataLoader for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [124]:
# Set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [125]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

/Users/revatishelat/opt/anaconda3/envs/PY396/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [126]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc="Training"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


Training:   0%|          | 0/1155 [00:00<?, ?it/s]

Training:   0%|          | 0/1155 [00:06<?, ?it/s]


IndexError: Target 3 is out of bounds.

In [ ]:
# Evaluation on validation set
model.eval()
val_preds = []
val_true = []
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Validation"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        
        val_preds.extend(preds)
        val_true.extend(labels.cpu().numpy())

# Evaluation metrics
print("Validation Accuracy:", accuracy_score(val_true, val_preds))
print("Classification Report:\n", classification_report(val_true, val_preds))        

In [ ]:
# Evaluation on test set
model.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        
        test_preds.extend(preds)
        test_true.extend(labels.cpu().numpy())

# Evaluation metrics
print("Test Accuracy:", accuracy_score(test_true, test_preds))
print("Classification Report:\n", classification_report(test_true, test_preds))
